In [ ]:
!pip install transformers

In [2]:
cd /content/drive/MyDrive/commit_folder/competition/tour_ai/code

/content/drive/MyDrive/commit_folder/competition/tour_ai/code


In [3]:
server = 'colab'
from PIL import Image
import numpy as np
from utils import dataset, models, utils
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [4]:
if server == 'colab':
  mecab_data_path = '/content/drive/MyDrive/commit_folder/competition/tour_ai/data/mecab_data.csv'
  model_save_dict = '/content/drive/MyDrive/commit_folder/competition/tour_ai/data/model/bert_weight.pt'
  test_data_path = '/content/drive/MyDrive/commit_folder/competition/tour_ai/data/mecab_test_data.csv'
  base_dir = '/content/drive/MyDrive/commit_folder/competition/tour_ai/data'
elif server == 'docker':
  mecab_data_path = '/tour_ai/data/mecab_data.csv'

In [5]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

image_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()
])

data, label_encoding = utils.data_preprocess(mecab_data_path)
data = utils.tokenize_sentence(data)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [6]:
data = data[['img_path','label']]

In [8]:
data['img_path'] = data['img_path'].apply(lambda x: x.replace('train','compress_train'))

In [9]:
train, valid = train_test_split(data, test_size=0.05, random_state=42)

In [10]:
train_dataset = dataset.ImageDataset(train, image_transforms, base_dir)

valid_dataset = dataset.ImageDataset(valid, image_transforms, base_dir)

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size = 16, shuffle=True)

In [12]:
a, b = next(iter(train_dataloader))

In [13]:
model = models.ImageModel(num_classes=len(label_encoding.classes_))

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

In [14]:
LR = 1e-5
EPOCHS = 1

In [15]:
utils.image_train(model, train_dataset, valid_data=valid_dataset, lr=LR, epochs=EPOCHS, is_valid=True)

  0%|          | 0/1009 [00:00<?, ?it/s]/content/drive/MyDrive/commit_folder/competition/tour_ai/code/utils/models.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
  1%|          | 9/1009 [00:55<1:43:38,  6.22s/it]


KeyboardInterrupt: ignored